<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/8_Module_2_Part_IV_Parallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture Module 2 Part 4: Parallelism

## LinkedIn Social Analysis

We now consider extensions of processing to multi-core and multi-machine models.  This includes:

* Polars, a multi-core Pandas-style library that is seeing wide adoption due to its performance (and flexible programming model)
* Sharded execution, a technique that is the basis of Spark's execution model.


In [ ]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

## PennGrader Setup

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
!pip3 install penngrader-client

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%set_env HW_ID=cis5450_25f_HW9

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

# Loading our Data

Now that we've seen how to do fairly complex queries over data in relations, we'll "pop back" to our big data example, which is the LinkedIn dataset.  Recall that we had a segment of the LinkedIn input file in our previous examples earlier in this module.

In [ ]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# Time conversions
import time

In [ ]:
'''
Simple code to pull out data from JSON and load into DuckDB.
'''
linked_in = open('linkedin_anon.jsonl')
people_df = pd.read_json('linkedin_anon.jsonl', lines=True)

In [ ]:
people_df

In [ ]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)

# Take the lists, drop any blank strings
specialties_df = people_df[['_id','specilities']].explode('specilities')
specialties_df.dropna(inplace=True)
interests_df = people_df[['_id','interests']].explode('interests')
interests_df.dropna(inplace=True)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

education_df = get_nested_list_dict(people_df[['_id','education']], 'education')
experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')
skills_df = get_nested_list(people_df[['_id','skills']], 'skills')
honors_df = get_nested_list(people_df[['_id','honors']], 'honors')
events_df = get_nested_list_dict(people_df[['_id','events']], 'events')

groups_df = get_nested_dict(people_df[['_id','group']], 'group')

people_only_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests']).merge(names_df,on='_id')

# Multi-Core Execution of Dataframes

## Polars

Polars is a "drop-in replacement" for Pandas that supports more advanced processing, including multicore processing.

Its syntax is not exactly the same as Pandas', but close.  (In fact, it has some similarities to Apache Spark DataFrames, which we'll be seeing shortly.)

### Polars DataFrames

To take advantage of Polars you want to convert your dataframes from Pandas to Polars. (If you have base files, you can also use Polars' read_csv etc as you would from Pandas.)

In [ ]:
import polars as pl

# Convert all Pandas dataframes to Polars Dataframes
people_only_pdf = pl.from_pandas(people_only_df)
education_pdf = pl.from_pandas(education_df)
experience_pdf = pl.from_pandas(experience_df)
skills_pdf = pl.from_pandas(skills_df)
groups_pdf = pl.from_pandas(groups_df)
names_pdf = pl.from_pandas(names_df)

## PL.Col

In Polars, when you are referring to a column in a DataFrame expression, this is represented by a *column object*.  For instance, you can express predicates on the values in a column when filtering, or you can request that certain columns be projected.

You can, e.g., talk about `pl.col('_id')`.  SQL-style, you can also refer to `pl.col('*')` if you are asking for all columns.

## Selection and Projection in Polars

Rather than using Pandas' `x[x[condition]]` notation, Polars has a *filter* function.   To project, you use a function called `select` (this is a bit confusing, but think SQL SELECT, not relational algebra select).

In [ ]:
people_only_pdf.\
  select(pl.col('_id','family_name','given_name')).\
  filter(pl.col('family_name') <= 'Smith')

## Joins

In many settings, multi-way joins are the most expensive query operations -- partly because they "blow up the data". You saw this with our discussions about join ordering.

We should be able to see the benefits of Polars multicore processing with some join expressions. Note that we still need to keep everything in memory for Pandas/Polars, and in fact there are simple join queries (say, all people joined with experiences and education) that will cause us to run out of memory.

Let's try a simpler query that joins people with some of their other info.

### Pandas Version

In [ ]:
%%time
people_only_df.merge(skills_df, on='_id').merge(education_df, on='_id').merge(groups_df, on='_id')

### Polars Version

Notice this finishes in roughly half the time as Pandas!

In [ ]:
%%time
print(people_only_pdf.join(skills_pdf, on='_id').join(education_pdf, on='_id').join(groups_pdf, on='_id'))

## Laziness

In [ ]:
%%time
result_pdf = people_only_pdf.join(skills_pdf, on='_id').\
  join(education_pdf, on='_id').join(groups_pdf, on='_id')
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf))

In [ ]:
%%time
result_pdf = people_only_pdf.lazy().join(skills_pdf.lazy(), on='_id').join(education_pdf.lazy(), on='_id').join(groups_pdf.lazy(), on='_id').lazy()
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf.collect()))

In [ ]:
results = only_sf_pdf.explain()

for line in results.split('\n'):
  print(line)

# Understanding Sharded Execution

Let's now look (in simulation) at how sharding works!  This is the basis of cluster-based processing.

## A Data Graph in Relations

To do this we'll adapt our LinkedIn data to a *graph* scenario, with people and companies.

In [ ]:
persons_df = people_only_df[['_id','family_name','given_name']]

persons_df

In [ ]:
experience_df

Companies are also nodes.  We'll auto-assign IDs for these.

In [ ]:
companies_df = experience_df[['org']].drop_duplicates()
companies_df['company_id'] = companies_df.index
companies_df

Finally, here are some edges.  We preserve the relationships between people and their experiences with organizations.

In [ ]:
worked_for = experience_df[['_id', 'org', 'title']].copy().merge(companies_df.set_index('org'), on='org').drop_duplicates()
worked_for = worked_for.drop(columns=['org'])
worked_for

### Data

In [ ]:
print('persons_df:')
display(persons_df)
print('companies_df:')
display(companies_df)
print('worked_df:')
display(worked_for)

### Implementing Sharded Computations - Our Operators

Rather than look at sharding in Spark, we'll instead simulate having 4 machines, each of which has access to a shard of each table.

To illustrate the basic concepts we'll create our own basic implementations of Relational Algebra over sharded data -- as well as our own `repartition` etc.

In [ ]:
nodes = 4

def get_hash(x):
  '''
  Hash code for data, used to compute a shard
  '''
  if isinstance(x, int):
    return x
  elif isinstance(x, str):
    return x.__hash__()
  elif pd.isna(x):
    return 'null'.__hash__()
  else:
    raise RuntimeError('Cannot hash {}'.format(type(x)))

def get_shard(sharded_df: list[pd.DataFrame], index: int):
  '''
  Retrieves the sub-dataframe corresponding to a shard
  '''
  return sharded_df[index]

def repartition(df_or_shards, key: str, shards: int = nodes):
  '''
  Take a dataframe or a list of sharded sub-dataframes.
  Repartition them on the key, based on the number of nodes.
  Return the new list of sub-dataframes.
  '''
  data = df_or_shards
  if isinstance(df_or_shards, list):
    data = pd.concat(df_or_shards)

  resharded_df = []
  for shard in range(0, shards):
    resharded_df.append(data[data[key].apply(lambda x: get_hash(x) % shards == shard)])

  return resharded_df

def rename_shards(sharded_df: list[pd.DataFrame], renamer: dict):
  '''
  Simple relational algebra operator:
  Rename the columns of a list of sharded dataframes
  '''
  renamed_df = []
  for shard in range(0, len(sharded_df)):
    renamed_df.append(sharded_df[shard].rename(columns=renamer))

  return renamed_df

def filter_shards(sharded_df: list[pd.DataFrame], filter_fn):
  '''
  Simple relational algebra operator:
  Filter a list of sharded dataframes with a predicate
  '''
  filtered_df = []
  for shard in range(0, len(sharded_df)):
    filtered_df.append(
        sharded_df[shard][
            sharded_df[shard].apply(filter_fn, axis=1)])

  return filtered_df


def join_shards(sharded_df1: list[pd.DataFrame], sharded_df2: list[pd.DataFrame],
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join two lists of sharded dataframes
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2[shard], left_on=left_key, right_on=right_key))

  return ret_result

def join_shard_with_broadcast(sharded_df1: list[pd.DataFrame], sharded_df2: pd.DataFrame,
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join a list of sharded dataframes with a single dataframe
  that conceptually gets "broadcast" to all of the shards
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2, left_on=left_key, right_on=right_key))

  return ret_result

## An Example Query

Suppose we simply want to ask this query:

`(persons_df) -[worked_for]--> (companies_df)`

## Shard the Relations!

We'll create versions of this:
* `sharded_persons` (by _id)
* `sharded_companies` (by company)
* Two shards of `worked_for`:
  - `sharded_by_user`
  - `sharded_by_company`

In [ ]:
# Generally, for a table like persons, we would
# shard by default on the key (_id).  We can
# see each shard here.

sharded_persons = repartition(persons_df, '_id')

for p in sharded_persons:
  display(p)

In [ ]:
# Similarly with companies... we shard by company_id
sharded_companies = repartition(companies_df, 'company_id')

for c in sharded_companies:
  display(c)

Edges are inherently trickier to shard: we can shard by the *source* or by the *destination* but can't simultaneously do so for a single table, by both.  We can, of course, create two dataframes - one sharded by each.

In [ ]:
# Edge sharded by user ID
sharded_by_user = repartition(worked_for, '_id')

for edge in sharded_by_user:
  display(edge)

Here's the other version...  Sharded by the company ID.

In [ ]:
sharded_by_companies = repartition(worked_for, 'company_id')

for edge in sharded_by_companies:
  display(edge)

## Let's Do Some Computation!

### Single Join: People -> works_for

To do this, we need to be careful to join only between tables that are sharded by the join key (_id).

In [ ]:
# sharded_persons is sharded by _id
# sharded_by_user is worked_for sharded by _id
for table in join_shards(sharded_persons, sharded_by_user, '_id', '_id'):
  display(table)

# Show the final results of the join
display(pd.concat(join_shards(sharded_persons, sharded_by_user, '_id', '_id')))

So, 256823 rows!

We can prove the above is correct by looking at the standard, unsharded result.

In [ ]:
# Traditional Pandas query, check the count and results
persons_df.merge(worked_for, left_on='_id', right_on='_id')

### What If We Don't Watch Our Sharding?

What if we don't align the shard columns and join keys? Let's see...

In [ ]:
# sharded_persons is sharded by _id
# sharded_by_companies is worked_for sharded by the company

# Output the result of the join without proper sharding
pd.concat(join_shards(sharded_persons, sharded_by_companies, '_id', '_id'))

What happens if we *broadcast* one of the relations?

We would generally only do this with a small table.

In [ ]:
pd.concat(join_shard_with_broadcast(sharded_persons, worked_for, '_id', '_id'))

## Now Let's Do a Full Connection

OK, we saw one join.  If we want to traverse our network from people to companies, we need two joins.

Here's the Pandas query we want to do in a sharded model...

In [ ]:
persons_df.merge(worked_for, left_on='_id', right_on='_id').merge(companies_df, left_on='company_id', right_on='company_id')

To join with `companies_df` we need to repartition our intermediate result on the company.

In [ ]:
# This is the join from above
persons_work_for = join_shards(sharded_persons, sharded_by_user, '_id', '_id')

# persons_work_for is sharded by _id
# sharded_companies is sharded by company
# to join them, we need to find a common sharding key -- the company

# SO: we repartition persons_work_for by company
persons_companies = join_shards(repartition(persons_work_for, 'company_id'), sharded_companies, 'company_id', 'company_id')

pd.concat(
persons_companies
)

Again, this matches our Pandas query.

## Two Hops

Can we find employees who have a common employer?

Let's divide it into steps.

First, let's do a building block, namely the set of people, companies they work for, and the interconnecting `works` relationships:

```
people --> works_for --> company <-- works_for
[---- (persons_companies) -----]
```

In [ ]:
# sharded_by_companies is the works_for edge, sharded by company
second_works_for = rename_shards(sharded_by_companies, {'_id': '_id2', 'role': 'role2'})

# We are currently sharded on company
person_works_company_works = \
    filter_shards(join_shards(persons_companies, second_works_for, 'company_id', 'company_id'),
           lambda row: row['_id'] != row['_id2']
    )
pd.concat(person_works_company_works)

Now we can join with another company of the

In [ ]:
sharded_persons_2 = rename_shards(sharded_persons, {'_id': '_id2', 'given_name': 'given_name2', 'last_name': 'last_name2'})

result = join_shards(repartition(person_works_company_works, '_id2'), sharded_persons_2, '_id2', '_id2')
# Take the above and shard + join with sharded_persons
for table in result:
  display(table)

display(pd.concat(result))

## Exercise

Let's do this via a broadcast join.

In [ ]:
# TODO
result = join_shard_with_broadcast(# TODO
)

display(pd.concat(result))

In [ ]:
grader.grade('sharded_net_result', len(pd.concat(result)))